## Calculate the effective diffusion temperature
<br>
  
#### Syntax
`HeliumPars = CalculateEDTsFxn(RunPars, HeliumPars)` <br>
#### Input 
`RunPars` : dictionary of terms consistent for the whole run of the model.<br>
`HeliumPars` : dictionary of terms relevant for He-3 production and diffusion . <br>
#### Variables Used
##### Helium Pars
`Hez` : depth vector of len(max_depths[0] / dz) <br>
`Ea` : activation energy <br>
`R` : R constant <br> 
#### Output
##### Helium Pars
`EDTs` : vector of size len(Hez) with spacing dz containing the effective diffusion temperature (essentially a scaled mean) (K).<br>
`Diff_z` : vector of size len(Hez) spaced dz with the diffusivity at every depth <br>
`DepthMeanTs` : vector size len(Hez) spaced dz with mean T at each depth <br>
#### Notes
**Date of Creation:** 7. Juli 2021 <br>
**Author:** Donovan Dennis <br>
**Update:** <br>

In [ ]:
def CalculateEDTsFxn(RunPars, HeliumPars):    
    # import parameters
    Hez = HeliumPars['Hez']
    Ea = HeliumPars['EDT_Ea']
    R = HeliumPars['R']
    nz = HeliumPars['nz']
    # run the crank-nicolson setup for the diffusion of temperature
    Temps, HeliumPars = ReturnBedrockTemps(RunPars, HeliumPars)
    # open empty lists
    depth_mean_Ts = [] * np.shape(Temps)[0]
    depth_integ = [] * np.shape(Temps)[0]
    # convert demps to EDTs
    exper = lambda t: np.exp(-Ea / (R * (t + 273.15)))
    exps = np.array([exper(Tempsi) for Tempsi in Temps])
    EDTs = [(-Ea / R / np.log(np.mean(exps[i,:]))) for i in range(nz)]
    # determing depth mean T   
    DepthMeanTs = [(np.mean(Temps[i,:]) + 273.15) for i in range(nz)]
    # calculate diffusivity for the EDTs
    Diff_z = diff_calc(EDTs)
    #assign outputs to dictionary
    HeliumPars['EDTs'] = EDTs
    HeliumPars['Diff_z'] = Diff_z
    HeliumPars['DepthMeanTs'] = DepthMeanTs
    
    return HeliumPars